In [188]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import os
import csv

In [189]:
mortality_with_sundown_census_poverty = pd.read_csv('../data/mortality_with_sundown_census_poverty.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [190]:
mortality_with_sundown_census_poverty = mortality_with_sundown_census_poverty.rename(
    columns={
        'All Ages in Poverty Percent': 'perc_poverty',
        'Median Household Income in Dollars': 'median_income_str'
    })

In [191]:
df_sundown = pd.read_csv('../data/sundown_with_counties.csv')
analysis_df = pd.merge(mortality_with_sundown_census_poverty, df_sundown, how='left', left_on='County', right_on="county")

In [192]:
# pull out state
analysis_df['state'] = analysis_df.County.apply(lambda x: x.split(', ')[1])

In [193]:
# merge in state shootings data
state_shootings = pd.read_csv('../data/police-killings-state.csv')
analysis_df = pd.merge(analysis_df, state_shootings, left_on='state', right_on='State Abbreviation', how='left')

In [194]:
# layer in jim crow states
jim_crow_states = [
    'TX', 'OK', 'MO', 'AR', 'LA', 'KY', 'WV', 'MD', 'VA', 'TN', 'NC', 'MS', 'AL', 'GA', 'SC', 'FL'
]
analysis_df['is_jim_crow_state'] = analysis_df.state.apply(lambda x: x in jim_crow_states)

In [195]:
# layer in state racism index
racism_index = pd.read_csv('../data/racism_index.csv')
analysis_df = pd.merge(analysis_df, racism_index, left_on='state', right_on='state_abbr', how='left')

In [196]:
# Merge in city shootings df
city_shootings = pd.read_csv('../data/police-killings-city.csv')
analysis_df = pd.merge(analysis_df, city_shootings, left_on='County', right_on='county', how='left')

In [200]:
analysis_df_with_city_shootings = analysis_df.dropna(subset=['Killings by Police per 10k Arrests'])

analysis_df_with_city_shootings['killings_per_10k'] = analysis_df_with_city_shootings['Killings by Police per 10k Arrests']
analysis_df_with_city_shootings['diss_index'] = analysis_df_with_city_shootings['Black-White Dissimilarity Index (2010)']
analysis_df_with_city_shootings['num_sundown_towns'] = analysis_df_with_city_shootings['#']
analysis_df_with_city_shootings['perc_white'] = 1 - analysis_df_with_city_shootings['perc_minority']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [201]:
analysis_df['median_income'] = analysis_df.median_income_str.apply(lambda x: x.replace('$', '').replace(',', ''))

# Analysis using full dataframe (full analysis_df)

In [202]:
# analysis using full dataframe

y = analysis_df['Death_Population_Ratio']
logit_covariates = [
    'perc_minority',
#     '#',
    'is_sundown_county',
    'perc_poverty',
    'is_jim_crow_state',
    'median_income',
]
X = analysis_df[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [128]:
logit_model = sm.Logit(y.astype(float), X.astype(float))
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.014561
         Iterations 9


In [129]:
result.summary2()

/usr/local/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:3390: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.llf/self.llnull


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
=======================================================================
Model:              Logit                  Pseudo R-squared: inf       
Dependent Variable: Death_Population_Ratio AIC:              11490.5376
Date:               2020-11-07 19:40       BIC:              11544.9608
No. Observations:   394212                 Log-Likelihood:   -5740.3   
Df Model:           4                      LL-Null:          0.0000    
Df Residuals:       394207                 LLR p-value:      1.0000    
Converged:          1.0000                 Scale:            1.0000    
No. Iterations:     9.0000                                             
------------------------------------------------------------------------
                    Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
------------------------------------------------------------------------
perc_minority       1.0910    0.1065   10.2403  0.0000   0.8822   1.2998
is_sundown_county  -0.0942    0.0375   -2.5155  0.0119  -0.1676  -0.0208
perc_poverty       -0.1117    0.0025  -43.8483  0.0000  -0.1167  -0.1067
is_jim_crow_state  -0.2980    0.0375   -7.9431  0.0000  -0.3715  -0.2245
median_income      -0.0000    0.0000  -78.6454  0.0000  -0.0000  -0.0000
=======================================================================

"""

# Analysis using just cities we have shootings data for

1. regress (minority%, # of sundown town, sundown county status, perc of poverty, jim crow state, median income) -> killings by police

In [178]:
y = analysis_df_with_city_shootings['killings_per_10k']
logit_covariates = [
    'perc_minority',
    '#',
    'is_sundown_county',
    'perc_poverty',
    'is_jim_crow_state',
    'median_income',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] == '':


In [179]:
ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()

In [180]:
ols_result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Ordinary least squares
=================================================================================
Model:                  OLS              Adj. R-squared (uncentered): 0.809      
Dependent Variable:     killings_per_10k AIC:                         592561.2394
Date:                   2020-11-07 20:14 BIC:                         592617.6736
No. Observations:       89832            Log-Likelihood:              -2.9627e+05
Df Model:               6                F-statistic:                 6.360e+04  
Df Residuals:           89826            Prob (F-statistic):          0.00       
R-squared (uncentered): 0.809            Scale:                       42.879     
------------------------------------------------------------------------------------
                        Coef.     Std.Err.       t       P>|t|      [0.025    0.975]
------------------------------------------------------------------------------------
perc_minority           5.8621      0.2461    23.8233    0.0000     5.3798    6.3444
#                       0.0694      0.0028    24.9947    0.0000     0.0639    0.0748
is_sundown_county       4.4734      0.0874    51.1929    0.0000     4.3022    4.6447
perc_poverty            0.2713      0.0057    47.7325    0.0000     0.2602    0.2825
is_jim_crow_state      -0.1017      0.0668    -1.5230    0.1277    -0.2327    0.0292
median_income           0.0000      0.0000    44.8435    0.0000     0.0000    0.0000
---------------------------------------------------------------------------------
Omnibus:                  4097.883           Durbin-Watson:              0.937   
Prob(Omnibus):            0.000              Jarque-Bera (JB):           6290.426
Skew:                     0.413              Prob(JB):                   0.000   
Kurtosis:                 3.999              Condition No.:              748801  
=================================================================================
* The condition number is large (7e+05). This might indicate             strong
multicollinearity or other numerical problems.
"""

2.  Regression with JIM Crow + Sundowntowns → Black White Dissimilarity Index (2010)

In [181]:
y = analysis_df_with_city_shootings['Black-White Dissimilarity Index (2010)']
logit_covariates = [
    'perc_minority',
    '#',
    'is_sundown_county',
    'perc_poverty',
    'is_jim_crow_state',
    'median_income',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] == '':


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                    Results: Ordinary least squares
=======================================================================================================
Model:                  OLS                                    Adj. R-squared (uncentered): 0.925      
Dependent Variable:     Black-White Dissimilarity Index (2010) AIC:                         750295.6376
Date:                   2020-11-07 20:14                       BIC:                         750352.0718
No. Observations:       89832                                  Log-Likelihood:              -3.7514e+05
Df Model:               6                                      F-statistic:                 1.842e+05  
Df Residuals:           89826                                  Prob (F-statistic):          0.00       
R-squared (uncentered): 0.925                                  Scale:                       248.21     
--------------------------------------------------------------------------------------------------------------
                            Coef.         Std.Err.           t            P>|t|          [0.025         0.975]
--------------------------------------------------------------------------------------------------------------
perc_minority              75.0784          0.5920        126.8175        0.0000        73.9180        76.2387
#                           0.2290          0.0067         34.2971        0.0000         0.2159         0.2421
is_sundown_county          10.3204          0.2102         49.0886        0.0000         9.9083        10.7325
perc_poverty                1.9655          0.0137        143.7056        0.0000         1.9387         1.9923
is_jim_crow_state           6.3968          0.1607         39.8017        0.0000         6.0818         6.7118
median_income              -0.0001          0.0000        -45.0087        0.0000        -0.0001        -0.0001
-------------------------------------------------------------------------------------------------------
Omnibus:                          9619.138                  Durbin-Watson:                     0.490   
Prob(Omnibus):                    0.000                     Jarque-Bera (JB):                  2783.905
Skew:                             0.084                     Prob(JB):                          0.000   
Kurtosis:                         2.154                     Condition No.:                     748801  
=======================================================================================================
* The condition number is large (7e+05). This might indicate             strong multicollinearity or
other numerical problems.
"""

3. regress (black-white diss. index(2010), minority%, # of sundown town, sundown county status, perc of poverty, jim crow state, median income) -> killings by police

In [182]:
y = analysis_df_with_city_shootings['Killings by Police per 10k Arrests']
logit_covariates = [
    'perc_minority',
    '#',
    'is_sundown_county',
    'perc_poverty',
    'is_jim_crow_state',
    'Black-White Dissimilarity Index (2010)',
    'median_income',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] == '':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  del sys.path[0]


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                  Results: Ordinary least squares
===================================================================================================
Model:                  OLS                                Adj. R-squared (uncentered): 0.811      
Dependent Variable:     Killings by Police per 10k Arrests AIC:                         591853.9356
Date:                   2020-11-07 20:14                   BIC:                         591919.7754
No. Observations:       89832                              Log-Likelihood:              -2.9592e+05
Df Model:               7                                  F-statistic:                 5.505e+04  
Df Residuals:           89825                              Prob (F-statistic):          0.00       
R-squared (uncentered): 0.811                              Scale:                       42.542     
-----------------------------------------------------------------------------------------------------
                                            Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
-----------------------------------------------------------------------------------------------------
perc_minority                               8.6295     0.2661    32.4252   0.0000    8.1079    9.1511
#                                           0.0778     0.0028    27.9646   0.0000    0.0724    0.0833
is_sundown_county                           4.8538     0.0882    55.0324   0.0000    4.6810    5.0267
perc_poverty                                0.3438     0.0063    54.7477   0.0000    0.3315    0.3561
is_jim_crow_state                           0.1340     0.0671     1.9971   0.0458    0.0025    0.2656
Black-White Dissimilarity Index (2010)     -0.0369     0.0014   -26.6844   0.0000   -0.0396   -0.0342
median_income                               0.0000     0.0000    40.5585   0.0000    0.0000    0.0000
---------------------------------------------------------------------------------------------------
Omnibus:                        3425.052                 Durbin-Watson:                    0.924   
Prob(Omnibus):                  0.000                    Jarque-Bera (JB):                 6129.276
Skew:                           0.314                    Prob(JB):                         0.000   
Kurtosis:                       4.115                    Condition No.:                    813496  
===================================================================================================
* The condition number is large (8e+05). This might indicate             strong multicollinearity
or other numerical problems.
"""

4. Add interaction term: white_perc * diss_index

In [183]:
ols_model = smf.ols(formula='killings_per_10k ~ 1 + perc_white*diss_index + num_sundown_towns + is_sundown_county + perc_poverty + is_jim_crow_state',
                   data=analysis_df_with_city_shootings)
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


<class 'statsmodels.iolib.summary2.Summary'>
"""
                       Results: Ordinary least squares
=============================================================================
Model:                 OLS                 Adj. R-squared:        0.147      
Dependent Variable:    killings_per_10k    AIC:                   583167.1069
Date:                  2020-11-07 20:14    BIC:                   583242.3525
No. Observations:      89832               Log-Likelihood:        -2.9158e+05
Df Model:              7                   F-statistic:           2217.      
Df Residuals:          89824               Prob (F-statistic):    0.00       
R-squared:             0.147               Scale:                 38.621     
-----------------------------------------------------------------------------
                           Coef.   Std.Err.    t     P>|t|   [0.025   0.975] 
-----------------------------------------------------------------------------
Intercept                  65.8639   0.6094 108.0835 0.0000  64.6695  67.0582
is_sundown_county[T.True]   2.4301   0.0886  27.4208 0.0000   2.2564   2.6038
is_jim_crow_state[T.True]  -1.6780   0.0671 -25.0077 0.0000  -1.8095  -1.5465
perc_white                -73.8067   0.7629 -96.7492 0.0000 -75.3019 -72.3115
diss_index                 -0.9644   0.0104 -92.5396 0.0000  -0.9849  -0.9440
perc_white:diss_index       1.2550   0.0141  88.7978 0.0000   1.2273   1.2827
num_sundown_towns           0.0738   0.0026  27.8846 0.0000   0.0686   0.0790
perc_poverty                0.1645   0.0073  22.4807 0.0000   0.1502   0.1789
-----------------------------------------------------------------------------
Omnibus:                3155.151          Durbin-Watson:             0.892   
Prob(Omnibus):          0.000             Jarque-Bera (JB):          7885.381
Skew:                   0.166             Prob(JB):                  0.000   
Kurtosis:               4.413             Condition No.:             3395    
=============================================================================
* The condition number is large (3e+03). This might indicate
strong multicollinearity or other numerical problems.
"""

5. Regression with JIM + Sundown towns + **percentage white+ Black White Dissimilarity Index (2010) → Racism index (2013-2017) in state racism index 2013-2017 file

In [203]:
ols_model = smf.ols(formula='racism_index ~ 1 + perc_white*diss_index + num_sundown_towns + is_sundown_county + perc_poverty + is_jim_crow_state',
                   data=analysis_df_with_city_shootings)
ols_result = ols_model.fit()
ols_result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                       Results: Ordinary least squares
==============================================================================
Model:                  OLS                 Adj. R-squared:        0.461      
Dependent Variable:     racism_index        AIC:                   618818.7050
Date:                   2020-11-07 20:17    BIC:                   618893.9506
No. Observations:       89832               Log-Likelihood:        -3.0940e+05
Df Model:               7                   F-statistic:           1.099e+04  
Df Residuals:           89824               Prob (F-statistic):    0.00       
R-squared:              0.461               Scale:                 57.435     
------------------------------------------------------------------------------
                           Coef.   Std.Err.     t     P>|t|   [0.025   0.975] 
------------------------------------------------------------------------------
Intercept                  46.4702   0.7431   62.5328 0.0000  45.0137  47.9267
is_sundown_county[T.True]  -0.6134   0.1081   -5.6752 0.0000  -0.8252  -0.4015
is_jim_crow_state[T.True] -14.3026   0.0818 -174.7897 0.0000 -14.4630 -14.1422
perc_white                 11.2589   0.9303   12.1024 0.0000   9.4355  13.0823
diss_index                  0.3806   0.0127   29.9450 0.0000   0.3557   0.4055
perc_white:diss_index      -0.2392   0.0172  -13.8792 0.0000  -0.2730  -0.2054
num_sundown_towns           0.2171   0.0032   67.2344 0.0000   0.2108   0.2234
perc_poverty               -0.7300   0.0089  -81.7914 0.0000  -0.7475  -0.7126
------------------------------------------------------------------------------
Omnibus:                 1877.850          Durbin-Watson:             0.090   
Prob(Omnibus):           0.000             Jarque-Bera (JB):          3215.763
Skew:                    0.184             Prob(JB):                  0.000   
Kurtosis:                3.851             Condition No.:             3395    
==============================================================================
* The condition number is large (3e+03). This might indicate
strong multicollinearity or other numerical problems.
"""